In [1]:
from eppy import modeleditor
import pythermalcomfort
import ladybug_comfort
import esoreader
import pandas as pd
import numpy as np
from numpy.random import randint
import openpyxl

import os
import sys
import glob
import tqdm

# Importando arquivos do EnergyPlus para realizar simução

In [2]:
IDF_PATH = '/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/klimaa/input_files/FAURB_50.idf'
IDD_PATH = '/usr/local/EnergyPlus-9-3-0/Energy+.idd'
EPW_PATH = '/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/klimaa/input_files/BRA_RS_Pelotas-2003-2017.epw'
OUTPUT_PATH = '/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/klimaa/simulacoes/output_files/round5'

In [3]:
modeleditor.IDF.setiddname(IDD_PATH) # Informando arquivo Energy +
building_idf = modeleditor.IDF(IDF_PATH, EPW_PATH) # Criando o arquivo idf com o arquivo climatico

/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/energy_plus_sm_env/lib/python3.10/site-packages/eppy/idfreader.py:268: ResourceWarning: unclosed file <_io.BufferedReader name='/usr/local/EnergyPlus-9-3-0/Energy+.idd'>
  versiontuple = iddversiontuple(iddfile)


In [4]:
dir(building_idf)
print(building_idf.idfobjects)

{'VERSION': [
Version,
    9.3;                      !- Version Identifier
], 'SIMULATIONCONTROL': [
SimulationControl,
    Yes,                      !- Do Zone Sizing Calculation
    No,                       !- Do System Sizing Calculation
    No,                       !- Do Plant Sizing Calculation
    Yes,                      !- Run Simulation for Sizing Periods
    Yes;                      !- Run Simulation for Weather File Run Periods
], 'PERFORMANCEPRECISIONTRADEOFFS': [], 'BUILDING': [
Building,
    Untitled,                 !- Name
    0,                        !- North Axis
    City,                     !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullInteriorAndExterior,    !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    ;                         !- Minimum Number of Warmup Days
], 'SHADOWCALCULATION': [], 'SURFACECONVECTION

In [5]:
building_idf.idfobjects["SCHEDULE:CONSTANT"][0].Hourly_Value = 0

In [6]:
help(building_idf)

Help on IDF in module eppy.modeleditor object:

class IDF(builtins.object)
 |  IDF(idfname=None, epw=None)
 |  
 |  The IDF class holds all the information about an EnergyPlus IDF.
 |  
 |  Attributes
 |  ----------
 |  iddname : str
 |      Name of the IDD currently being used by eppy. As a class attribute, this
 |      is set for all IDFs which are currently being processed and cannot be
 |      changed for an individual IDF.
 |  iddinfo : list
 |      Comments and metadata about fields in the IDD.
 |  block : list
 |      Field names in the IDD.
 |  
 |  
 |  
 |  idfname : str
 |      Path to the IDF file.
 |  idfobjects : list
 |      List of EpBunch objects in the IDF.
 |  model : Eplusdata object
 |      Data dictionary and list of objects for the entire model.
 |  outputtype : str
 |      How to format the output of IDF.print or IDF.save, IDF.saveas or
 |      IDF.savecopy. The options are: 'standard', 'nocomment', 'nocomment1',
 |      'nocomment2', and 'compressed'.
 |  
 |  

# Simulando execução (Energy Plus)

Arquivos gerados serão salvos na pasta "~\Projeto Confortímetro KLIMAA\arquivosep\saidasep"

In [7]:
for vel_ar in range(0, 16):
    building_idf.idfobjects["SCHEDULE:CONSTANT"][0].Hourly_Value = vel_ar/10
    building_idf.save()
    if not os.path.exists(os.path.join(OUTPUT_PATH, str(vel_ar))):
        os.makedirs(os.path.join(OUTPUT_PATH, str(vel_ar)))
    building_idf = modeleditor.IDF(IDF_PATH, EPW_PATH)
    print(f"Velocidade ar: {building_idf.idfobjects['SCHEDULE:CONSTANT'][0].Hourly_Value}")
    os.system(f"/usr/local/EnergyPlus-9-3-0/energyplus --weather {EPW_PATH} --output-directory {os.path.join(OUTPUT_PATH, str(vel_ar))} --idd {IDD_PATH} --expandobjects {IDF_PATH}")

/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/energy_plus_sm_env/lib/python3.10/site-packages/eppy/idfreader.py:268: ResourceWarning: unclosed file <_io.BufferedReader name='/usr/local/EnergyPlus-9-3-0/Energy+.idd'>
  versiontuple = iddversiontuple(iddfile)


Velocidade ar: 0.0
ExpandObjects Started.
 Begin reading Energy+.idd file.
 Done reading Energy+.idd file.
ExpandObjects Finished. Time:     0.055
EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2023.09.08 17:12
Initializing Response Factors
Calculating CTFs for "EXTERIOR FLOOR", Construction # 1
Calculating CTFs for "EXTERIOR WALL", Construction # 3
Calculating CTFs for "INTERIOR WALL", Construction # 4
Calculating CTFs for "EXTERIOR ROOF", Construction # 5
Calculating CTFs for "EXTERIOR DOOR", Construction # 9
Calculating CTFs for "PAREDE_GESSO", Construction # 11
Calculating CTFs for "PISO_VINILICO", Construction # 12
Calculating CTFs for "BASE_COBERTURA", Construction # 13
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializi

EnergyPlus Completed Successfully.
/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/energy_plus_sm_env/lib/python3.10/site-packages/eppy/idfreader.py:268: ResourceWarning: unclosed file <_io.BufferedReader name='/usr/local/EnergyPlus-9-3-0/Energy+.idd'>
  versiontuple = iddversiontuple(iddfile)


Velocidade ar: 0.1
ExpandObjects Started.
 Begin reading Energy+.idd file.
 Done reading Energy+.idd file.
ExpandObjects Finished. Time:     0.056
EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2023.09.08 17:13
Initializing Response Factors
Calculating CTFs for "EXTERIOR FLOOR", Construction # 1
Calculating CTFs for "EXTERIOR WALL", Construction # 3
Calculating CTFs for "INTERIOR WALL", Construction # 4
Calculating CTFs for "EXTERIOR ROOF", Construction # 5
Calculating CTFs for "EXTERIOR DOOR", Construction # 9
Calculating CTFs for "PAREDE_GESSO", Construction # 11
Calculating CTFs for "PISO_VINILICO", Construction # 12
Calculating CTFs for "BASE_COBERTURA", Construction # 13
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializi

EnergyPlus Completed Successfully.
/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/energy_plus_sm_env/lib/python3.10/site-packages/eppy/idfreader.py:268: ResourceWarning: unclosed file <_io.BufferedReader name='/usr/local/EnergyPlus-9-3-0/Energy+.idd'>
  versiontuple = iddversiontuple(iddfile)


...

EnergyPlus Completed Successfully.
/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/energy_plus_sm_env/lib/python3.10/site-packages/eppy/idfreader.py:268: ResourceWarning: unclosed file <_io.BufferedReader name='/usr/local/EnergyPlus-9-3-0/Energy+.idd'>
  versiontuple = iddversiontuple(iddfile)


Velocidade ar: 1.4
ExpandObjects Started.
 Begin reading Energy+.idd file.
 Done reading Energy+.idd file.
ExpandObjects Finished. Time:     0.062
EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2023.09.08 17:27
Initializing Response Factors
Calculating CTFs for "EXTERIOR FLOOR", Construction # 1
Calculating CTFs for "EXTERIOR WALL", Construction # 3
Calculating CTFs for "INTERIOR WALL", Construction # 4
Calculating CTFs for "EXTERIOR ROOF", Construction # 5
Calculating CTFs for "EXTERIOR DOOR", Construction # 9
Calculating CTFs for "PAREDE_GESSO", Construction # 11
Calculating CTFs for "PISO_VINILICO", Construction # 12
Calculating CTFs for "BASE_COBERTURA", Construction # 13
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializi

EnergyPlus Completed Successfully.
/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/energy_plus_sm_env/lib/python3.10/site-packages/eppy/idfreader.py:268: ResourceWarning: unclosed file <_io.BufferedReader name='/usr/local/EnergyPlus-9-3-0/Energy+.idd'>
  versiontuple = iddversiontuple(iddfile)


Velocidade ar: 1.5
ExpandObjects Started.
 Begin reading Energy+.idd file.
 Done reading Energy+.idd file.
ExpandObjects Finished. Time:     0.060
EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2023.09.08 17:28
Initializing Response Factors
Calculating CTFs for "EXTERIOR FLOOR", Construction # 1
Calculating CTFs for "EXTERIOR WALL", Construction # 3
Calculating CTFs for "INTERIOR WALL", Construction # 4
Calculating CTFs for "EXTERIOR ROOF", Construction # 5
Calculating CTFs for "EXTERIOR DOOR", Construction # 9
Calculating CTFs for "PAREDE_GESSO", Construction # 11
Calculating CTFs for "PISO_VINILICO", Construction # 12
Calculating CTFs for "BASE_COBERTURA", Construction # 13
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializi

EnergyPlus Completed Successfully.


##  Lendo as saídas geradas
Variaveis a serem utilizadas como parametro pelas funções do Pythermal

- tdb : temperatura do ar seco -> Site Outdoor Air Drybulb Temperature

- tr : temperatura radiante media ->  Zone Mean Radiant Temperature

- vr : velocidade relativa do ar -> Schedule Value: Velocidade

- rh : humidade relativa do ar -> People Air Relative Humidity OU Zone Air Relative Humidity

- met : taxa metabolica -> 1.2

- clo : isolamento de roupas -> Zone Thermal Comfort Clothing Value

- wme : trabalho externo (ZERO) -> 0

Inicializando a variavel eso com o arquido .eso gerado após a simulação

In [5]:
eso = esoreader.read_from_path(f"{OUTPUT_PATH}/eplusout.eso")
teste = eso.to_frame("PEOPLE_SALA_AULA")
print(teste)

Empty DataFrame
Columns: []
Index: []


In [23]:
print(eso.to_frame("PEOPLE_SALA_AULA"))

Empty DataFrame
Columns: []
Index: []


In [16]:
print()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
var = eso.find_variable("")
print(var[0])
    

('TimeStep', 'Environment', 'Site Outdoor Air Drybulb Temperature')


In [11]:
def get_room_info(eso, room_name:str="SALA_AULA"):
    data = pd.DataFrame(pd.date_range("2017-01-01", "2018-01-02 T23:45", freq="15T"))
    humidade_relativa = pd.DataFrame()
    humidade_relativa.ffill()
    ocupacao = pd.DataFrame()
    ocupacao.ffill()
    clothing = pd.DataFrame()
    clothing.ffill()
    pmv_ep = pd.DataFrame()
    pmv_ep.ffill()
    ashrae_55 = pd.DataFrame()
    ashrae_55.ffill()
    
    results = None
    
    temp_externa = eso.to_frame("Site Outdoor Air Drybulb Temperature")["Environment"]
    temp_interna = eso.to_frame("Zone Air Temperature")[room_name]
    mrt = eso.to_frame("Zone Mean Radiant Temperature")[room_name]
    humidade = eso.to_frame("Zone Air Relative Humidity")[room_name]
    velocidade_ar = eso.to_frame('Schedule Value')[f"VEL_{room_name}"]
    
    try:
        humidade_relativa = eso.to_frame("People Air Relative Humidity")[f"People_{room_name.lower()}"]
        ocupacao = eso.to_frame("People Occupant Count")[f"People_{room_name.lower()}"]
        clothing = eso.to_frame('Zone Thermal Comfort Clothing Value')[f"People_{room_name.lower()}"]
        pmv_ep = eso.to_frame("Zone Thermal Comfort Fanger Model")[f"People_{room_name.lower()}"]
        ashrae_55 = eso.to_frame("Zone Thermal Comfort ASHRAE 55 Adaptive Model Temperature")[f"People_{room_name.lower()}"]
    except Exception as ex:
        results = pd.concat([temp_externa, temp_interna, humidade, velocidade_ar], axis=1)
        results.columns = ['temp_externa', 'temp_interna', 'humidade', 'velocidade_ar']
        print(f"Couldn't get {ex}")
        return results
        
    results = pd.concat([data, temp_externa, temp_interna, humidade, humidade_relativa, velocidade_ar, ocupacao, clothing, pmv_ep, ashrae_55], axis=1)
    results.columns = ['data', 'temp_externa', 'temp_interna', 'humidade', 'humidade_relativa', 'velocidade_ar', 'ocupacao', 'clothing', 'pmv_ep', 'ashrae_55']
    return results

In [3]:
rooms = [
    "SALA_AULA",
    "ATELIE1",
    "ATELIE2",
    "ATELIE3",
    "LINSE",
    "SEC_LINSE",
    "RECEPCAO"
]

In [18]:
eso_files = glob.glob(f"{OUTPUT_PATH}/*.eso")
eso = esoreader.read_from_path(eso_files[0])
pmv = eso.to_frame("Zone Thermal Comfort Fanger Model PMV")["PEOPLE_SALA_AULA"]
vel = eso.to_frame('Schedule Value')
print(pmv)
print(vel)
#pmv.to_excel("teste_pmv.xlsx")

0       -4.678173
1       -4.681053
2       -4.683952
3       -4.686915
4       -4.689904
           ...   
35227   -0.957593
35228   -0.964477
35229   -0.971446
35230   -0.978431
35231   -0.985437
Name: PEOPLE_SALA_AULA, Length: 35232, dtype: float64
Empty DataFrame
Columns: []
Index: []


In [12]:
met = 1.15 # metabolic rate
wme = 0   # external work, normally around 0

eso_files = glob.glob(f"{OUTPUT_PATH}/*.eso")
group_table = {}
filter_ocupacao = False

for eso_file in tqdm.tqdm(eso_files):
    eso = esoreader.read_from_path(eso_file)
    #eso_index = int(eso_file.split(os.sep)[-2])
    
    for room in rooms:
        table = get_room_info(eso, room)
        """
        if filter_ocupacao:
            table = table[table['ocupacao']>0]
        
        
        for index, row in table.iterrows():
            pmv_pythermal = pythermalcomfort.models.pmv(
                row['temp_interna'],
                row['temp_externa'],
                row['velocidade_ar'],
                row['humidade_relativa'],
                met,
                row['clothing'],
                wme,
                standard='ashrae',
                limit_inputs=False
            )

            pmv_ladybug = ladybug_comfort.pmv.predicted_mean_vote(
                row['temp_interna'],
                row['temp_externa'],
                row['velocidade_ar'],
                row['humidade_relativa'],
                met,
                row['clothing'],
                wme
            )

            table.loc[index, 'pmv_pythermal'] = pmv_pythermal
            table.loc[index, 'pmv_ladybug'] = pmv_ladybug['pmv']
        """ 
        if not room in group_table:
            group_table.update({room: table})
        else:
            group_table[room] = pd.concat([group_table[room], table], axis=0, ignore_index=True)

  0%|                                                     | 0/1 [00:03<?, ?it/s]


KeyError: 'VEL_SALA_AULA'

In [29]:
for room in rooms:
    group_table[room]['data'] = pd.to_datetime(group_table[room]['data'])
    temp = group_table[room][(group_table[room].pmv_pythermal >= -1.1) & (group_table[room].pmv_pythermal <= 1.1)]
    min_pmv = temp.groupby('data')['velocidade_ar'].idxmin()
    temp = temp.loc[min_pmv]

    min_outter_bound = group_table[room][(group_table[room].velocidade_ar == 0.0) & (group_table[room].pmv_pythermal < -1.1)]
    max_outter_bound = group_table[room][(group_table[room].velocidade_ar == 1.5) & (group_table[room].pmv_pythermal > 1.1)]

    group_table[room] = pd.concat([temp, max_outter_bound, min_outter_bound], ignore_index=True)
    group_table[room]['status_pmv'] = group_table[room]["pmv_pythermal"].apply(lambda x: "OK" if x <= 1.1 and x >= -1.1 else "NÃO OK")

In [30]:
for room, table in group_table.items():
    table.to_excel(os.path.join(OUTPUT_PATH, f"{room}.xlsx"), sheet_name=room)
    print(f"{room}.xlsx saved")

SALA_AULA_ladybug.xlsx saved
ATELIE1_ladybug.xlsx saved
ATELIE2_ladybug.xlsx saved
ATELIE3_ladybug.xlsx saved
LINSE_ladybug.xlsx saved
SEC_LINSE_ladybug.xlsx saved
RECEPCAO_ladybug.xlsx saved


In [19]:
test = group_table[rooms[0]]
test['data'] = pd.to_datetime(test['data'])

test3 = test[(test.pmv_ep >= -1.1) & (test.pmv_ep <= 1.1)]
min_pmv = test3.groupby('data')['velocidade_ar'].idxmin()
test3 = test3.loc[min_pmv]

min_outter_bound = test[(test.velocidade_ar == 0.0) & (test.pmv_ep < -1.1)]
max_outter_bound = test[(test.velocidade_ar == 1.5) & (test.pmv_ep > 1.1)]

test = pd.concat([test3, max_outter_bound, min_outter_bound], ignore_index=True)
test['status_pmv'] = test["pmv_ep"].apply(lambda x: "OK" if x <= 1.1 and x >= -1.1 else "NÃO OK")

In [20]:
test.to_excel(os.path.join(OUTPUT_PATH, f"ATELIE1-4.xlsx"))

In [7]:
excel_files = glob.glob("/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/output_files/round2/*.xlsx")

for excel_file in tqdm.tqdm(excel_files):
    table = pd.read_excel(excel_file)
    for index, config in table.iterrows():
        try:
            diff = config["pmv_ep"] - config["PMV Pythermal"]
            table.loc[index, 'Diff PMV EP and PMV Pythermal'] = diff
        except Exception as ex:
            print(f"Couldn't calculate pmv for {room} : {ex}")
            break
    table.to_excel(excel_file)
    print(f"File {excel_file} saved")

 14%|██████▎                                     | 1/7 [05:06<30:38, 306.39s/it]

File /mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/output_files/round2/ATELIE1.xlsx saved


 29%|████████████▌                               | 2/7 [09:50<24:26, 293.25s/it]

File /mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/output_files/round2/ATELIE2.xlsx saved


 43%|██████████████████▊                         | 3/7 [14:53<19:51, 297.89s/it]

...

File /mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/output_files/round2/RECEPCAO.xlsx saved


 86%|█████████████████████████████████████▋      | 6/7 [31:25<05:20, 320.21s/it]

File /mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/output_files/round2/SALA_AULA.xlsx saved


100%|████████████████████████████████████████████| 7/7 [37:04<00:00, 317.78s/it]

File /mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/output_files/round2/SEC_LINSE.xlsx saved


In [12]:
excel_file = "/mnt/sda1/gabriellb/Documentos/Faculdade/projetos/gaia/ProjFAURB/confortimetro_klimaa/output_files/round2/ATELIE1.xlsx"
table = pd.read_excel(excel_file)

In [13]:
print(table)

                Unnamed: 0  temp_externa  temp_interna   humidade  \
0      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
1      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
2      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
3      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
4      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
...                    ...           ...           ...        ...   
563707 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563708 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563709 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563710 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563711 2018-01-02 23:45:00       18.6000     25.583168  38.010608   

        humidade_relativa  velocidade_ar  ocupacao  clothing    pmv_ep  \
0               75.594521            0.0         0  0.750000 -3.464261   
1               75.5945

In [14]:
table = table.rename({"Unnamed: 0":"Data"}, axis="columns")

In [15]:
print(table)

                      Data  temp_externa  temp_interna   humidade  \
0      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
1      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
2      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
3      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
4      2017-01-01 00:00:00        6.0934     10.163171  75.633097   
...                    ...           ...           ...        ...   
563707 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563708 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563709 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563710 2018-01-02 23:45:00       18.6000     25.583168  38.010608   
563711 2018-01-02 23:45:00       18.6000     25.583168  38.010608   

        humidade_relativa  velocidade_ar  ocupacao  clothing    pmv_ep  \
0               75.594521            0.0         0  0.750000 -3.464261   
1               75.5945

In [20]:
temp_index = None
temp_date = None
temp_pmv_ep = None
for index, row in tqdm.tqdm(table.iterrows()):
    if row["Data"] != temp_date or temp_date == None:
        temp_index = index
        temp_date = row["Data"]
        temp_pmv_ep = row["pmv_ep"]
        continue
        
    if row["pmv_ep"] < -1.1 or row["pmv_ep"]:
        table.drop([index])
        continue
        
    if row["pmv_ep"] < temp_pmv_ep:
        table.drop([temp_index])

4776it [02:17, 34.80it/s]


KeyboardInterrupt: 

In [ ]:
print(table)

In [ ]:
table.to_excel(excel_file)

In [122]:
results[results['ocupacao']>0]

,ocupacao,temp_interna,clothing,pmv_ep,velocidade
120,29.0,26.457370,0.0,1.087710,0.0
121,29.0,26.486482,0.0,0.886912,0.0
122,29.0,26.525749,0.0,0.884262,0.0
123,29.0,26.549846,0.0,0.896900,0.0
124,29.0,26.566620,0.0,0.906535,0.0
...,...,...,...,...,...
35103,2.9,24.724794,0.0,0.631601,0.0
35104,2.9,24.716255,0.0,0.630244,0.0
35105,2.9,24.705925,0.0,0.628464,0.0
35106,2.9,24.691216,0.0,0.625430,0.0


In [127]:
results[(results['pmv_ep']>0.5) & (results['ocupacao']>0)]

,ocupacao,temp_interna,clothing,pmv_ep,velocidade
120,29.0,26.457370,0.0,1.087710,0.0
121,29.0,26.486482,0.0,0.886912,0.0
122,29.0,26.525749,0.0,0.884262,0.0
123,29.0,26.549846,0.0,0.896900,0.0
124,29.0,26.566620,0.0,0.906535,0.0
...,...,...,...,...,...
35103,2.9,24.724794,0.0,0.631601,0.0
35104,2.9,24.716255,0.0,0.630244,0.0
35105,2.9,24.705925,0.0,0.628464,0.0
35106,2.9,24.691216,0.0,0.625430,0.0


In [85]:
# temperatura do ar - externa
temp_externa = eso.to_frame('Site Outdoor Air Drybulb Temperature') # Variavel correta -> Zone Outdoor Air Drybulb

# temp. do ar - interna
temp_interna = eso.to_frame('Zone Mean Radiant Temperature')

# Humidade relativa
humidade_relativa = eso.to_frame('Zone Air Relative Humidity')

# Clothing
clothing = eso.to_frame('Zone Thermal Comfort Clothing Value')

# Velocidade do ar interno
velocidade_ar = eso.to_frame('VELOCIDADE')

# pmv calculado pelo Energy Plus
pmv_ep = eso.to_frame('Zone Thermal Comfort Fanger Model PMV')

In [50]:
metrics = list(metrics)
results = eso.to_frame(metrics[0])
for t in metrics[1:]:
    results.append(eso.to_frame(t))
results

AttributeError: 'DataFrame' object has no attribute 'append'

In [45]:
data = pd.DataFrame()

data.columns = ['ocupacao','temp_externa', 'temp_interna', 'humidade_relativa', 'clothing', 'velocidade_ar', 'pmv_ep']
#data.velocidade_ar = data.velocidade_ar.fillna(0) # Inicializando a velocidade do ar com 0 para todas situações
# rodar na simulacao
data

""


In [ ]:
data = data.tail(-50).copy() # Removendo as 50 primeiras linhas, contendo lixo gerado pelo Energy Plus
data.reset_index(drop=True, inplace=True)
data

,ocupacao,temp_externa,temp_interna,humidade_relativa,clothing,velocidade_ar,pmv_ep
0,0.0,21.1,31.430514,100.000000,0.750000,0,2.906900
1,0.0,21.0,31.175821,100.000000,0.750000,0,2.819135
2,0.0,21.1,30.945120,100.000000,0.750000,0,2.734382
3,0.0,21.6,30.725895,100.000000,0.750000,0,2.650895
4,0.0,22.8,30.661125,99.902235,0.485311,0,2.428023
...,...,...,...,...,...,...,...
8753,23.0,20.0,30.882852,98.819061,0.511823,0,2.565419
8754,23.0,19.6,30.841801,100.000000,0.511823,0,3.033489
8755,23.0,19.2,30.457939,100.000000,0.511823,0,2.678953
8756,0.0,18.9,30.124419,100.000000,0.511823,0,2.662846


# Definição de funções auxiliares

In [ ]:
# Definindo o status do conforto térmico através do PMV resultante
def pmv_status(pmv):
    if pmv >= 1.1 :
        return 'quente'
    elif pmv <= -1.1:
        return 'frio'
    else:
        return 'ok'

# Calculando PMV atavés da PythermalComfort

pythermalcomfort.models.pmv(tdb, tr, vr, rh, met, clo, wme=0, standard='ISO', **kwargs)

In [ ]:
for index, config in data.iterrows():
    pmvresultado = round(pmv(config['temp_externa'],
                    config['temp_interna'],
                    config['velocidade_ar'],
                    config['humidade_relativa'],
                    met,
                    clo_dynamic(config['clothing'], met),
                    wme,
                    standard='ashrae', limit_inputs=False), 1)

    data.loc[index, 'PMV Pythermal'] = pmvresultado
    data.loc[index, 'PMV Pythermal Status'] = pmv_status(pmvresultado)

In [ ]:
data['velocidade_ar'].unique()

array([0], dtype=int64)

### Recalculando o PMV, fazendo alterarções na velocidade do ar com base no pmv anterior

In [ ]:
for index, config in data.iterrows():
        param_vel_ar = round(data.loc[index, 'velocidade_ar'], 2)
        #print(param_vel_ar)

        pmvresultado = round(pmv(config['temp_externa'],
                        config['temp_interna'],
                        param_vel_ar,
                        config['humidade_relativa'],
                        met,
                        clo_dynamic(config['clothing'], met),
                        wme,
                        standard='ashrae', limit_inputs=False), 1)

        if pmv_status(pmvresultado) == 'quente': # pvm maior que 1
            if param_vel_ar < 1: # velocidade do ar menor que 1
                data.loc[index+1:, 'velocidade_ar'] += 0.1
                #print('aumentou')

        elif pmv_status(pmvresultado) == 'frio': # pvm menor que -1
            if 0 < param_vel_ar : # velocidade do ar maior que 0
                data.loc[index+1:, 'velocidade_ar'] = 0
                #print('diminuiu')


        data.loc[index, 'PMV Mod'] = pmvresultado
        data.loc[index, 'PMV Mod Status'] = pmv_status(pmvresultado)


In [ ]:
data.head(50)
# tirar ocupacao 0

,ocupacao,temp_externa,temp_interna,humidade_relativa,clothing,velocidade_ar,pmv_ep,PMV Pythermal,PMV Pythermal Status,PMV Mod,PMV Mod Status
0,0.0,21.1,31.430514,100.000000,0.750000,0.0,2.906900,1.0,ok,1.0,ok
1,0.0,21.0,31.175821,100.000000,0.750000,0.0,2.819135,0.9,ok,0.9,ok
2,0.0,21.1,30.945120,100.000000,0.750000,0.0,2.734382,0.9,ok,0.9,ok
3,0.0,21.6,30.725895,100.000000,0.750000,0.0,2.650895,1.0,ok,1.0,ok
4,0.0,22.8,30.661125,99.902235,0.485311,0.0,2.428023,0.8,ok,0.8,ok
5,0.0,24.1,30.729587,99.399393,0.485311,0.0,2.422281,1.1,quente,1.1,quente
6,23.0,25.3,31.042623,98.973668,0.485311,0.1,2.486470,1.4,quente,1.4,quente
7,23.0,26.4,31.622032,100.000000,0.485311,0.2,3.088079,1.7,quente,1.4,quente
8,23.0,27.1,32.149713,100.000000,0.485311,0.3,3.232665,1.9,quente,1.3,quente
9,23.0,27.6,32.647978,100.000000,0.485311,0.4,3.384355,2.1,quente,1.2,quente


In [ ]:
print(data['PMV Pythermal Status'].value_counts())
print(data['PMV Pythermal'].describe())


PMV Pythermal Status
ok        3800
frio      3603
quente    1355
Name: count, dtype: int64
count    8758.000000
mean       -0.615517
std         1.533233
min        -6.400000
25%        -1.700000
50%        -0.700000
75%         0.500000
max         4.100000
Name: PMV Pythermal, dtype: float64


In [ ]:
print(data['PMV Mod Status'].value_counts())
print(data['PMV Mod'].describe())

PMV Mod Status
ok        4339
frio      3716
quente     703
Name: count, dtype: int64
count    8758.000000
mean       -0.820153
std         1.332619
min        -6.400000
25%        -1.700000
50%        -0.800000
75%         0.100000
max         3.900000
Name: PMV Mod, dtype: float64


In [ ]:
ar_ligado = data[data['velocidade_ar']!=0]
ar_ligado['velocidade_ar'].describe()

count    2105.000000
mean        0.456057
std         0.293226
min         0.100000
25%         0.200000
50%         0.400000
75%         0.700000
max         1.000000
Name: velocidade_ar, dtype: float64

In [ ]:
data[data['PMV Mod Status'] == 'frio']

,ocupacao,temp_externa,temp_interna,humidade_relativa,clothing,velocidade_ar,pmv_ep,PMV Pythermal,PMV Pythermal Status,PMV Mod,PMV Mod Status
22,0.0,18.7,32.067724,100.000000,0.485311,0.7,3.102669,0.2,ok,-1.8,frio
44,0.0,19.6,32.224847,99.994676,0.504104,0.5,3.364739,0.4,ok,-1.1,frio
68,0.0,20.6,32.034898,99.995614,0.496501,0.6,3.306083,0.6,ok,-1.1,frio
93,0.0,22.2,32.574962,99.998730,0.479446,1.0,3.310170,1.0,ok,-1.1,frio
118,0.0,21.6,33.139848,100.000000,0.477265,1.0,3.499488,0.9,ok,-1.3,frio
...,...,...,...,...,...,...,...,...,...,...,...
8712,0.0,12.3,28.280988,100.000000,0.538964,0.0,1.647448,-1.5,frio,-1.5,frio
8713,0.0,12.0,27.870635,99.965210,0.538964,0.0,1.455794,-1.6,frio,-1.6,frio
8714,0.0,12.3,27.516733,100.000000,0.538964,0.0,1.305698,-1.6,frio,-1.6,frio
8715,0.0,13.3,27.195046,99.992975,0.538964,0.0,1.174056,-1.5,frio,-1.5,frio


In [ ]:
sala_ocupada = data[data['ocupacao']!=0]
sala_ocupada['PMV Mod Status'].value_counts()

PMV Mod Status
ok        2125
frio      1332
quente     558
Name: count, dtype: int64

# Calculo do comforto adaptativo

In [ ]:
t_running_mean = randint(20, 25, 7)
t_running_mean

array([21, 22, 23, 23, 21, 20, 20])

### adaptive_ashrae(tdb, tr, t_running_mean, v, units='SI', limit_inputs=True)

- tdb (float or array-like) – dry bulb air temperature, default in [°C] in [°F] if units = ‘IP’
- tr (float or array-like) – mean radiant temperature, default in [°C] in [°F] if units = ‘IP’
- t_running_mean (float or array-like) – running mean temperature, default in [°C] in [°C] in [°F] if units = ‘IP’
- v (float or array-like) – air speed, default in [m/s] in [fps] if units = ‘IP’
- units ({‘SI’, ‘IP’}) – select the SI (International System of Units) or the IP (Imperial Units) system.
- limit_inputs (boolean default True) – By default, if the inputs are outsude the standard applicability limits the function returns nan. If False returns pmv and ppd values even if input values are outside the applicability limits of the model.

In [ ]:
adaptive_ashrae(20, 30, running_mean_outdoor_temperature(t_running_mean, 0.8, units='SI'), 0.6, units='SI', limit_inputs=False)

{'tmp_cmf': 24.5,
 'tmp_cmf_80_low': 21.0,
 'tmp_cmf_80_up': 28.0,
 'tmp_cmf_90_low': 22.0,
 'tmp_cmf_90_up': 27.0,
 'acceptability_80': array(True),
 'acceptability_90': array(True)}